In [30]:
import fitz  # PyMuPDF para PDF
import docx
import os

def extract_text_from_pdf(pdf_path):
    """Extrae texto de un archivo PDF."""
    with fitz.open(pdf_path) as doc:
        return "\n".join([page.get_text("text") for page in doc])

def extract_text_from_docx(docx_path):
    """Extrae texto de un archivo DOCX."""
    doc = docx.Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_txt(txt_path):
    """Lee texto desde un archivo TXT."""
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read()

def extract_text(file_path):
    """Detecta el tipo de archivo y extrae su contenido."""
    _, ext = os.path.splitext(file_path)
    if ext.lower() == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext.lower() == ".docx":
        return extract_text_from_docx(file_path)
    elif ext.lower() == ".txt":
        return extract_text_from_txt(file_path)
    else:
        raise ValueError("Formato de archivo no soportado. Usa PDF, DOCX o TXT.")

# Prueba con un archivo
if __name__ == "__main__":
    file_path = "../01.01 - Análisis y diseño de algoritmos (2024-2) (1).pdf"  # Cambia esto con un archivo de prueba
    text = extract_text(file_path)
    print(text[:500])  # Muestra los primeros 500 caracteres del contenido


Análisis y diseño de algoritmos
(3009430)

Objetivo del curso (oficial)
Cualquiera con una experiencia mínima en algoritmos y 
programación sabe que para un mismo problema pueden 
existir innumerables algoritmos de solución. Tales 
soluciones pueden variar en diversos aspectos, siendo de 
particular interés la eficiencia en términos de su costo 
computacional. 
En el contexto de dicha eficiencia se puede hablar de 
algoritmos "mejores" que otros, siendo el objetivo principal 
de esta asignatura 


In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Cargar la API key desde .env
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

In [32]:
def generate_content(prompt):
    """Genera contenido con Gemini a partir de un prompt."""
    model = genai.GenerativeModel("gemini-2.0-flash-001")
    response = model.generate_content(prompt)
    return response.text

def generate_study_materials(course_text):
    """Genera notas de clase, problemas, preguntas, objetivos y lecturas con Gemini."""
    
    prompts = {
        "notas_clase": f"Genera notas de clase detalladas basadas en este programa de curso:\n{course_text}",
        "problemas_practica": f"Genera problemas de práctica con soluciones basados en este programa de curso:\n{course_text}",
        "preguntas_discusion": f"Genera preguntas para discusión relacionadas con los temas de este programa de curso:\n{course_text}",
        "objetivos_aprendizaje": f"Genera objetivos de aprendizaje detallados basados en este programa de curso:\n{course_text}",
        "lecturas_sugeridas": f"Sugiere lecturas y recursos académicos basados en este programa de curso:\n{course_text}"
    }

    materials = {key: generate_content(prompt) for key, prompt in prompts.items()}
    
    return materials

# Prueba con el texto extraído
if __name__ == "__main__":
    file_path = "../01.01 - Análisis y diseño de algoritmos (2024-2) (1).pdf"  # Reemplaza con tu archivo
    course_text = extract_text(file_path)
    materials = generate_study_materials(course_text)

    for key, content in materials.items():
        print(f"\n=== {key.replace('_', ' ').title()} ===\n{content[:1000]}...\n")



=== Notas Clase ===
¡Absolutamente! Aquí tienes unas notas de clase detalladas basadas en el programa del curso, diseñadas para capturar los puntos clave y proporcionar una estructura para el aprendizaje:

**Análisis y Diseño de Algoritmos (3009430)**

**Introducción**

*   **¿Por qué este curso?**
    *   Muchos algoritmos posibles para el mismo problema.
    *   Objetivo: Aprender a diseñar y analizar algoritmos eficientes (en costo computacional).

*   **Objetivos del curso (oficiales)**
    *   Ofrecer una asignatura optativa para Ingeniería de Sistemas y otros programas.
    *   Mejorar las habilidades de programación de los estudiantes.
    *   Preparar para maratones de programación y competencias.
    *   Aumentar las oportunidades laborales en tecnología (empresas o emprendimiento).

*   **Objetivos del curso (extraoficiales)**
    *   Comprender conceptos y problemas en diversas áreas (redes, criptografía, optimización, etc.).
    *   Desarrollar habilidades avanzadas de pro

In [29]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def generate_pdf(materials, output_filename="material_didactico.pdf"):
    """Genera un PDF con los materiales didácticos."""
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter

    c.setFont("Helvetica-Bold", 12)
    c.drawString(100, height - 50, "Material Didáctico Generado")

    y_position = height - 80
    c.setFont("Helvetica", 12)

    for section, content in materials.items():
        # Agregar título de la sección
        c.setFont("Helvetica-Bold", 14)
        c.drawString(100, y_position, section.replace("_", " ").title())
        y_position -= 20
        c.setFont("Helvetica", 12)

        # Dividir contenido en líneas para evitar desbordamiento
        for line in content.split("\n"):
            if y_position < 50:  # Salto de página si no hay espacio
                c.showPage()
                c.setFont("Helvetica", 12)
                y_position = height - 50
            c.drawString(100, y_position, line[:100])  # Truncar líneas largas
            y_position -= 15

    c.save()
    print(f"PDF generado: {output_filename}")

# Prueba con los materiales generados
if __name__ == "__main__":
    generate_pdf(materials)

PDF generado: material_didactico.pdf
